In [1]:
from sloyka import VkCommentsParser, VkPostGetter

/home/gk/.cache/pypoetry/virtualenvs/factfinder-6OvYRBkP-py3.10/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


[nltk_data] Downloading package stopwords to /home/gk/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
owner_id = "-60463565" # группа "девяткино онлайн"
token = '492f35ec492f35ec492f35ecdd4a3926724492f492f35ec2c17df616b6994a83242f520'

In [3]:
post_num_limit = 10
step = 10
#Вызов метода
post_ids = VkPostGetter.get_group_post_ids(owner_id, token, post_num_limit, step)

print(len(post_ids))

10


In [4]:
comments_num = 30

# Вызов метода
result = VkCommentsParser.get_Comments(post_ids, owner_id, token, comments_num=comments_num)

 40%|████      | 4/10 [00:16<00:24,  4.01s/it]


In [7]:
result.sample(3)

,name,last_name,date,likes,text,post_id
2672542,Yashka,Artillerist,2024-02-13,1,У нас народ выберет как обычно он хороший чело...,2672516
2672606,Alexandra,Petrova,2024-02-13,7,"Три года вместе, а что не женишься?\nДевушка п...",2672582
2672593,Nadezhda,Nikitina,2024-02-13,1,Я с помощью этого бота смогла узнать с кем мой...,2672582
